# Data discovery: Load and Join

In [1]:
import logging # used for saving error messages to a file.
from pyspark.sql import SparkSession

## Create Spark Session. Remember this is our entry point for Dataframes or Datasets, not RDDs! 

- **SparkContext** : It is the entry point for using RDD (Resilient Distributed Datasets) as the underlying data structure in Apache Spark, especially in older versions of Spark. But in newer versions of Spark, such as Spark 2.0 and above, the use of SparkContext is no longer necessary with SparkSession. A wrapper for handling RDDs, DataFrames, Datasets, SQL queries, and other operations in Spark.

- **SparkSession** : It is an entry point for working with DataFrames, Datasets, and SQL queries in Apache Spark. It integrates management with all Spark resources. This makes it more convenient and intuitive to work with columnar data. (columnar-structured data) and working with structured data (structured data) in the form of DataFrames and Datasets.

In [2]:
# Create a SparkSession
spark = SparkSession.builder.appName("RDDToDataFrame").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/08/20 13:58:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/08/20 13:58:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:

def rdd_to_dataframe(data, schema):
    try:
        # Create an RDD from the input data, using Spark Context not Session!
        rdd = spark.sparkContext.parallelize(data)

        # Convert RDD to DataFrame
        df = spark.createDataFrame(rdd, schema)

        # Return the DataFrame, without stopping the SparkSession
        return df

    except Exception as e:
        # Log error and Stop the SparkSession
        logging.error('Error while transforming RDD to DF: {}'.format(e))
        spark.stop()


In [4]:
# Create data
# Data dept sample
dept_data = [(1,"Big Data"), (2, "Finance"), (3,"Marketing")]
dept_schema = ["department_id", "department_name"]

# Data emp sample
emp_data = [(1,"Carlos", 17), (1,"Bob", 30), (2,"Jasmin", 26)]
emp_schema = ["department_id","employee_name", "age"]

## Use the Spark RDD as a Spark Dataframe

In [5]:
# Call function
df_emp = rdd_to_dataframe(emp_data, emp_schema)
df_dept = rdd_to_dataframe(dept_data, dept_schema)

In [6]:
# Show schema
df_dept.show()

+-------------+---------------+
|department_id|department_name|
+-------------+---------------+
|            1|       Big Data|
|            2|        Finance|
|            3|      Marketing|
+-------------+---------------+



In [7]:
df_emp.printSchema()

root
 |-- department_id: long (nullable = true)
 |-- employee_name: string (nullable = true)
 |-- age: long (nullable = true)



## Use Spark SQL, to join 2 datasets

In [8]:
# Register as view
df_emp.createOrReplaceTempView('employees')
df_dept.createOrReplaceTempView('departments')

In [9]:
# Query sample, using Spark SQL
spark.sql('''
            select emp.*, dept.*
            from employees as emp
                inner join departments as dept on (emp.department_id = dept.department_id) 
            where age >= 18
            ''').show()

+-------------+-------------+---+-------------+---------------+
|department_id|employee_name|age|department_id|department_name|
+-------------+-------------+---+-------------+---------------+
|            1|          Bob| 30|            1|       Big Data|
|            2|       Jasmin| 26|            2|        Finance|
+-------------+-------------+---+-------------+---------------+



In [10]:
# Let's now save the JOINED RESULTSET into a new Temporary View -- NO WHERE CLAUSE
spark.sql('''
        select emp.employee_name, emp.age, emp.department_id, dept.department_name
        from employees as emp
            inner join departments as dept on (emp.department_id = dept.department_id)
             where age >= 18
        ''').createOrReplaceTempView('dept_employees')

In [11]:
# Let's now save the JOINED RESULTSET into a new Temporary View -- NO WHERE CLAUSE
spark.sql('''
        select * from dept_employees where department_id is not null
        ''').show()

+-------------+---+-------------+---------------+
|employee_name|age|department_id|department_name|
+-------------+---+-------------+---------------+
|          Bob| 30|            1|       Big Data|
|       Jasmin| 26|            2|        Finance|
+-------------+---+-------------+---------------+



##  Save this output 

In [12]:
# Define output location
output_location = './output/dept_employees/'

# Save the result to local storage
spark.sql('''
        select * from dept_employees where department_id is not null
        ''').write.mode('append').csv(output_location)

In [13]:
spark.stop()